In [1]:
import fiona
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# Open Street Map Extract for Demographic Search
`civicknowledge.com-osm-demosearch-1.2.1` Last Update: 2021-02-15T17:38:34

__




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [geohash_tags](data/geohash_tags.csv)**. Points converted to counts of tags per geohash
* ** [residential_roads](data/residential_roads.csv)**. Residential roads per 4 digit geohash
* ** [nonres_roads](data/nonres_roads.csv)**. Non residential roads per 4 digit geohash
* ** [block_geo](data/block_geo.csv)**. Geographic shapes for blocs in block_osm
* ** [block_osm](data/block_osm.csv)**. Points from geohash_tags allocated to 2020 census blocks
* ** [utm_map](data/utm_map.csv)**. Map from census blocks to UTM zones
* ** [cbsa_map](data/cbsa_map.csv)**. Map from census blocks to CBSA Metros
* ** [bus_densities](notebooks/PointDensity.ipynb#bus_densities)**. High-density business goehashes

## References
<ul><li> <strong>us_geohashes</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-geohash-us.csv#us_geohashes</em>. All 4 digit geohases in the continential US</li><li> <strong><a href="https://download.geofabrik.de/north-america-latest.osm.pbf">north-america-latest</a></strong>. OSM North America extract</li><li> <strong>utm_grid</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-mgrs.csv#utm_grid</em>. </li><li> <strong>points</string>, <em>data/csv/points.csv</em>. Points from the OSM file</li><li> <strong>lines</string>, <em>data/csv/lines.csv</em>. Lines from the OSM file</li><li> <strong>multipolygons</string>, <em>data/csv/multipolygons.csv</em>. Polygons from the OSM file</li><li> <strong>multilinestrings</string>, <em>data/csv/multilinestrings.csv</em>. Lines from the OSM file</li><li> <strong>other_relations</string>, <em>data/csv/other_relations.csv</em>. Other geo data from the OSM file</li><li> <strong>block_templ</string>, <em>censusgeo://2020/5/{st}/block</em>. Block url template</li><li> <strong>cbsa</string>, <em>censusgeo://2020/5/US/cbsa</em>. Metro areas</li><ul>

In [4]:
%time blocks = pkg.resource('block_geo').geoframe()

CPU times: user 2min 14s, sys: 6.3 s, total: 2min 20s
Wall time: 2min 21s


In [5]:
%time osm_blocks = pkg.resource('block_osm').dataframe()

CPU times: user 1.21 s, sys: 223 ms, total: 1.44 s
Wall time: 1.44 s


In [7]:
%time utm_map = pkg.resource('utm_map').dataframe()
%time  cbsa_map = pkg.resource('cbsa_map').dataframe()

CPU times: user 7.18 s, sys: 1.34 s, total: 8.52 s
Wall time: 8.6 s
CPU times: user 5.59 s, sys: 885 ms, total: 6.47 s
Wall time: 6.54 s


In [27]:
t = utm_map[['geoid','band', 'epsg']]\
    .merge(cbsa_map.rename(columns={'block':'geoid'}), how='left')

SyntaxError: unexpected character after line continuation character (<ipython-input-27-a6def5368d42>, line 1)

In [20]:
np.sum(~t.cbsa.isnull())

6784756

In [26]:
ob_index = osm_blocks[['geoid']].merge(t)

,geoid,band,epsg,cbsa
0,10100US010010201001001,16,32616,31000US33860
1,10100US010010201001015,16,32616,31000US33860
2,10100US010010201001018,16,32616,31000US33860
3,10100US010010201001021,16,32616,31000US33860
4,10100US010010201001023,16,32616,31000US33860
...,...,...,...,...
741647,10100US560459513002031,13,32613,31000US23940
741648,10100US560459513002051,13,32613,31000US23940
741649,10100US560459513002062,13,32613,31000US23940
741650,10100US560459513002064,13,32613,31000US23940


In [32]:
ob_index.groupby('cbsa').count().sort_values('geoid',ascending=False)

,geoid,band,epsg
cbsa,,,
31000US35620,24970,24970,24970
31000US16980,15005,15005,15005
31000US31080,12445,12445,12445
31000US47900,10257,10257,10257
31000US37980,10137,10137,10137
...,...,...,...
31000US14420,33,33,33
31000US11380,25,25,25
31000US49820,21,21,21


In [66]:
from demosearch.search import ftsearch
from demosearch import FileCache
fc = FileCache('/Volumes/SSD_Extern/radius')
geoid = '31000US41740' #ftsearch(fc, 'Diego')[0].geoid

t = blocks.merge(ob_index[ob_index.cbsa == geoid], how='right')
t = t[['geoid', 'geometry']].merge(osm_blocks)
t['business'] = t.shop+t.bank+t.bar+t.cafe+t.clothes+t.convenience+t.bank+t.fast_food\
                +t.restaurant+t.supermarket
t['entertain'] = t.bar+t.cafe+t.convenience+t.bank+t.fast_food+t.restaurant
t[t.business+t.entertain > 0].to_csv('business_blocks.csv')